In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph import add_messages, StateGraph, END
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage
from dotenv import load_dotenv
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

load_dotenv()

sqlite_conn = sqlite3.connect("checkpoint.sqlite", check_same_thread=False)

In [2]:
# memory = MemorySaver()
memory = SqliteSaver(sqlite_conn)

llm = ChatGroq(
    model="llama-3.1-8b-instant"
)

In [3]:
class BasicChatState(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: BasicChatState):
    return {
        "messages": [llm.invoke(state["messages"])]
    }

In [4]:
graph = StateGraph(BasicChatState)

graph.add_node("chatbot", chatbot)
graph.set_entry_point("chatbot")
graph.add_edge("chatbot", END)

app = graph.compile(checkpointer = memory)

In [7]:
config = {
    "configurable": {
        "thread_id": 1
    }
}

while True: 
    user_input = input("User: ")
    if(user_input.lower() in ["exit", "end"]):
        break
    else: 
        result = app.invoke({
            "messages": [HumanMessage(content=user_input)]
        }, config=config)

        print("AI: " + result["messages"][-1].content)

AI: You are studying at Carnegie Mellon University (CMU). 

You are studying a Master of Science (M.S.) in Computational Data Science.
